# Testing the loading of saved onnx models

# Imports and Setup

Imports and colab setup

In [1]:
%%capture import_capture --no-stder
# Jupyter magic methods
# For auto-reloading when external modules are changed
%load_ext autoreload
%autoreload 2
# For showing plots inline
%matplotlib inline

import numpy as np

!pip install onnx
import onnx
!pip install onnxruntime
import onnxruntime as ort

In [2]:
# Give the notebook access to the rest of your google drive files.
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
# Enter the relevant foldername
FOLDERNAME = '/content/drive/My Drive/ML/arxiv_vixra'
assert FOLDERNAME is not None, "[!] Enter the foldername."
# For importing modules stored in FOLDERNAME or a subdirectory thereof:
import sys
sys.path.append(FOLDERNAME)

Mounted at /content/drive


Copy to cwd.

In [3]:
model_file_name = 'model_test.onnx'
FOLDERNAME = '/content/drive/My Drive/ML/arxiv_vixra'
file_path = FOLDERNAME + '/onnx_testing/' + model_file_name
!cp '{file_path}' .
!ls

drive  model_test.onnx	sample_data


In [4]:
# Load the ONNX model
model = onnx.load(model_file_name)

# Check that the model is well formed
onnx.checker.check_model(model)

# Print a human readable representation of the graph
print(onnx.helper.printable_graph(model.graph))

graph torch-jit-export (
  %input[FLOAT, batch_sizex128x69]
  %hiddens_in[FLOAT, 4xbatch_sizex128]
) initializers (
  %class_layer.weight[FLOAT, 1x256]
  %class_layer.bias[FLOAT, 1]
  %320[FLOAT, 2x768]
  %321[FLOAT, 2x384x69]
  %322[FLOAT, 2x384x128]
  %363[FLOAT, 2x768]
  %364[FLOAT, 2x384x256]
  %365[FLOAT, 2x384x128]
) {
  %20 = Constant[value = <Scalar Tensor []>]()
  %21 = Sub(%input, %20)
  %22 = Constant[value = <Scalar Tensor []>]()
  %23 = Div(%21, %22)
  %24 = Transpose[perm = [1, 0, 2]](%23)
  %141 = Constant[value = <Tensor>]()
  %142 = Constant[value = <Tensor>]()
  %143 = Constant[value = <Tensor>]()
  %144 = Slice(%hiddens_in, %142, %143, %141)
  %145, %146 = GRU[direction = 'bidirectional', hidden_size = 128, linear_before_reset = 1](%24, %321, %322, %320, %, %144)
  %147 = Transpose[perm = [0, 2, 1, 3]](%145)
  %148 = Constant[value = <Tensor>]()
  %149 = Reshape(%147, %148)
  %265 = Constant[value = <Tensor>]()
  %266 = Constant[value = <Tensor>]()
  %267 = Constant[

In [5]:
# Generate inputs with a random batch size
rand_batch_size = np.random.randint(512)
print(rand_batch_size)
input, hiddens = np.random.randn(rand_batch_size, 128 ,69).astype('float32'), np.zeros((4, rand_batch_size, 128)).astype('float32')

46


In [6]:
ort_session = ort.InferenceSession(model_file_name)
input_name = ort_session.get_inputs()
print(*[name for name in input_name])

NodeArg(name='input', type='tensor(float)', shape=['batch_size', 128, 69]) NodeArg(name='hiddens_in', type='tensor(float)', shape=[4, 'batch_size', 128])


In [7]:
outputs = ort_session.run(
    None,
    {"input": input,
     "hiddens_in": hiddens},
)

`outputs` is a list with two items, the 0th being the predictions and 1st the hidden state, both as np arrays.

In [8]:
outputs[0].shape

(46, 1)

In [9]:
outputs[1].shape

(4, 46, 128)

In [10]:
print(*[num[0] for num in outputs[0]], sep='\n')

0.839252
0.5225534
0.16992316
0.547248
0.44211
0.6753504
0.15221435
0.5701543
0.20706213
0.63829345
0.46372685
0.512088
0.9202295
0.5477316
0.1389465
0.35095865
0.7156574
0.55098486
0.83804
0.0749405
0.9580704
0.46807528
0.24682608
0.62955797
0.5819962
0.83355176
0.77640796
0.5709648
0.8646306
0.35684633
0.31753388
0.86338913
0.5572058
0.8963237
0.7962342
0.2397049
0.3705038
0.38047975
0.7124896
0.8800446
0.14302641
0.22292218
0.51515603
0.4354192
0.6529306
0.81278396
